In [1]:
import os
import sys
src_dir = os.path.join(os.getcwd(), '..', 'src')
sys.path.append(src_dir)

from utils import GLOBAL, functions

In [2]:
import pandas as pd

In [3]:
year = 2020

data_df = pd.read_csv(GLOBAL.CLEANED_DATA_2014_2019).iloc[:, 1:]

WEEKLY_BASE_URL = "https://raw.githubusercontent.com/fantasydatapros/data/master/weekly/{year}/week{week}.csv"

In [7]:
#Get 2020 Data and concat to the main data_df
for week in range(1, 18):
    weekly_df = pd.read_csv(WEEKLY_BASE_URL.format(year=year, week=week))
    weekly_df['season'] = int(year)
    weekly_df['week'] = week
    data_df = pd.concat([data_df, weekly_df])

In [9]:
data_df.head()

,player_id,tm,pos,player,season,week,NFL_draft_year,NFL_draft_round,NFL_draft_rank,height_inches,...,rec_yds,rec_td,rec,targets,rush_att,rush_td,rush_yds,fumble,fumble_lost,interceptions
0,2755,NYG,WR,Rueben Randle,2014,9,2012.0,2.0,63.0,74.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2755,NYG,WR,Rueben Randle,2014,10,2012.0,2.0,63.0,74.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2755,NYG,WR,Rueben Randle,2014,11,2012.0,2.0,63.0,74.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2755,NYG,WR,Rueben Randle,2014,12,2012.0,2.0,63.0,74.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2755,NYG,WR,Rueben Randle,2014,13,2012.0,2.0,63.0,74.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
